Copyright (C) 2017 The University of Sydney, Australia
This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License, version 2, as published by the Free Software Foundation.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program; if not, write to Free Software Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301, USA.

Authors: John Cannon and Simon Williams

### This notebook extracts subduction zone and mid-ocean ridge segments from the tecontic reconstruction model and samples them at regularly spaced intervals. For each point on the sz segment plate age and velocity are extracted. For each point on the mor segment velocity is extracted.

##### Input:

GPlates reconstructed topological plate boundaries

Sea floor age grids

##### Output:

file of time-dependent subduction zone kinematics and plate ages e.g. szData_summaryStats_NNRPlateModel.csv

file of  time-dependent mor kinematics e.g. morData_summaryStats_NNRPlateModel.csv


In [ ]:
from __future__ import print_function
import math
import os
import sys
sys.path.insert(1, '/Users/ajy321/pygplates_rev18_python27_MacOS64')
import pygplates
# Add directory containing the 'ptt' module (Plate Tectonic Tools) to the Python path.
import pandas as pd
import math
import matplotlib.pyplot as plt
import scipy.interpolate as spi
import numpy as np
os.chdir('/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/PlateTectonicTools')
import subduction_convergence
import ridge_spreading_rate


root = '/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/'

# Input rotation and topology files.
rotation_filename = [root+'1000-410_rotations-NNR.rot', 
                     root+'Global_EB_250-0Ma_GK07_2017-NNR.rot', 
                     root+'Global_EB_410-250Ma_GK07_2017-NNR.rot', 
                     root+'NR_0Ma_1000Ma_for_gplates.rot']

rotation_model = pygplates.RotationModel(rotation_filename)

topology_filenames = [root+'1000-410-Convergence-NNR.gpml', 
                      root+'1000-410-Divergence-NNR.gpml', 
                      root+'1000-410-Topologies-NNR.gpml', 
                      root+'1000-410-Transforms-NNR.gpml', 
                      root+'Global_EarthByte_Mesozoic-Cenozoic_plate_boundaries_2016_v5-NNR.gpml', 
                      root+'Global_EarthByte_Paleozoic_plate_boundaries_2016_v5-NNR.gpml', 
                      root+'TopologyBuildingBlocks_AREPS-NNR.gpml']

# Output file containing results at each reconstruction time.
output_filename = 'output.txt'

# Base filename and extension of raster to sample.
raster_filename_base ='/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/AgeGrids/agegrid_masked'
raster_filename_ext = 'nc'

# Define the time range.
# The reconstruction time range (topologies resolved to these times).
# Also used to get paleo raster filenames based on 'raster_filename_base'.
min_time = 0
max_time = 580
time_step = 20
# Tessellate the subduction zones to 0.5 degrees.
tessellation_threshold_radians = math.radians(0.5)

# name the model what you want.
model='NNRPlateModel'

# set working directory and make if it does not exist
workDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/DeepwaterCycling"
if not os.path.exists(workDir):
    os.makedirs(workDir)

In [ ]:
# This is a program to sample grids
# The method used here for sampling of masked (NaN) rasters is faster than using the 'raster_query' module.

try:
    from netCDF4 import Dataset as netcdf
except ImportError:
    from scipy.io import netcdf_file as netcdf
    print('Warning: NetCDF4 grids not supported ("netCDF4" Python module not found). '
          'Falling back to NetCDF3, rasters may fail to load.', file=sys.stderr)

import scipy.interpolate as spi
import numpy as np

def sample_grid_using_scipy(x,y,grdfile):
    
    data=netcdf(grdfile,'r')

    try:
        lon = np.copy(data.variables['x'][:])
        lat = np.copy(data.variables['y'][:])
    except:
        lon = np.copy(data.variables['lon'][:])
        lat = np.copy(data.variables['lat'][:])
    
    Zg = data.variables['z'][:]
        
    test = fill_ndimage(Zg)
    
    lut=spi.RectBivariateSpline(lon,lat,test.T)
    result = []
    for xi,yi in zip(x,y):
        result.append(lut(xi, yi)[0][0])
            
    return result


from scipy import ndimage as nd

def fill_ndimage(data,invalid=None):
    """Replace the value of invalid 'data' cells (indicated by 'invalid')
    by the value of the nearest valid data cell
    Parameters
    ----------
    data: numpy array of any dimension
    invalid: a binary array of same shape as 'data'. True cells set where data
    value should be replaced.
    If None (default), use: invalid = np.isnan(data)
    Returns
    -------
    Return a filled array.
    Credits
    -------
    http://stackoverflow.com/a/9262129
    """
    if invalid is None: invalid = np.isnan(data)
    ind = nd.distance_transform_edt(invalid, return_distances=False, return_indices=True)
    return data[tuple(ind)]


In [ ]:
# Extract subduction zone statistics
# Each element of this list will be a tuple of values for a specific reconstruction time.
output_data = []

# Create an empty dataframe to concatenate results to
df_AllTimes = pd.DataFrame()

# Reconstruction times.
reconstruction_times = np.arange(min_time, max_time, time_step)
    
# Iterate over time steps
for reconstruction_time in reconstruction_times:
    print (reconstruction_time)
    # Use existing subduction convergence script to generate sample points along subduction zones at 'time'.
    subduction_data = []
    subduction_convergence_data = subduction_convergence.subduction_convergence(
                rotation_model,
                topology_filenames,
                tessellation_threshold_radians,
                reconstruction_time)
    
    # Determine paleo raster filename.
    raster_filename = '{0}_{1}.{2}'.format(raster_filename_base, reconstruction_time, raster_filename_ext)
    
    # decompress newer GMT 5 grids
    grdDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/AgeGrids/"
    grdCLASSIC=grdDir+"agegrid_masked_%s.nc" %(reconstruction_time)
    
    if not os.path.isfile(grdCLASSIC):
        cmd="grdsample %s -R0/360/-90/90 -I0.1d -G%s --IO_NC4_CHUNK_SIZE=classic" %(raster_filename, grdCLASSIC)
        print (cmd)
        os.system(cmd)
    
    # Sample raster/grid at subduction points.
    subduction_lons = [data[0] for data in subduction_convergence_data]
    subduction_lats = [data[1] for data in subduction_convergence_data]
    conv_rate = [data[2] for data in subduction_convergence_data]
    conv_obliq = [data[3] for data in subduction_convergence_data]
    arc_length = [data[6] for data in subduction_convergence_data]
    plate_age = sample_grid_using_scipy(subduction_lons, subduction_lats, grdCLASSIC)

    # Make a flat list of subduction stats to input into the proximity test
    df = pd.DataFrame({'lon' : subduction_lons, 'lat': subduction_lats,
                      'conv_obliq' : conv_obliq, 'conv_rate' : conv_rate, 'arc_length' : arc_length,
                      'plate_age' : plate_age, 'reconstruction_time' : reconstruction_time})
    
    # append dataframe 
    df_AllTimes = df_AllTimes.append(df)

# export data
df_AllTimes.to_csv(workDir+'/szData_summaryStats_%s.csv' %(model))

#### Copyright (C) 2017 The University of Sydney, Australia
This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License, version 2, as published by the Free Software Foundation.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program; if not, write to Free Software Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301, USA.

Author: John Cannon

## Calculate average seafloor spreading rates of mid-ocean ridges over a sliding time window (over time)
The output file contains the following columns:

 reconstruction_time (Ma)
 total ridge length in metres (excluding transform segments)
 average spreading rate in cm/year (over 'averaging_time_window' million years)
 standard deviation of spreading rate in cm/year (over 'averaging_time_window' million years)

The spreading rates are only calculated along ridge sections (transform sections are excluded). Note that this algorithm only works well under the following conditions:

    - ridge segments are perpendicular to their spreading directions
    - isochron geometries are up-to-date with respect to the rotation model
            -ie, stage pole is in correct location relative to geometry
    -there are valid rotations (in rotation model) for each isochron at its birth time plus one
            -ie, 1My prior to isochron birth time
    -all isochrons have conjugate plate IDs

In [ ]:
spreading_feature_types = [pygplates.FeatureType.gpml_mid_ocean_ridge]
threshold_sampling_distance_radians = math.radians(0.5)

# Create an empty dataframe to concatenate results to
MORData_df = pd.DataFrame()

print('Calculating spreading rates from {0} to {1} Ma...'.format(min_time, max_time))

# Each element of this list will be a tuple of values for a specific reconstruction time.
output_data = []

# Read the topology filenames once instead of at every time step.
topology_features = []
for topology_filename in topology_filenames:
    topology_features.extend(pygplates.FeatureCollection(topology_filename))

# Keep track of spreading-rate-related quantities over time so can later calculate statistics.
total_length_metres_time_sequence = []
total_spreading_rate_times_length_time_sequence = []
total_spreading_rate_squared_times_length_time_sequence = []

# Iterate over times in inclusive range [min_time, max_time + averaging_time_window - 1].
for reconstruction_time in range(min_time, max_time, time_step):
    
    # Use existing ridge spreading rate script to generate sample points along mid-ocean ridges at 'time'.
    ridge_spreading_rate_data = ridge_spreading_rate.spreading_rates(
        rotation_model,
        topology_features,
        reconstruction_time,
        threshold_sampling_distance_radians,
        spreading_feature_types)
    
    # Sample at ridge points.
    ridge_lons = [data[0] for data in ridge_spreading_rate_data]
    ridge_lats = [data[1] for data in ridge_spreading_rate_data]
    spreading_rate = [data[2] for data in ridge_spreading_rate_data]
    arc_length = [data[3] for data in ridge_spreading_rate_data]
    
    # Make a flat list of subduction stats to input into the proximity test
    df = pd.DataFrame({'lon' : ridge_lons, 'lat': ridge_lats,
                      'spreading_rate' : spreading_rate, 'arc_length' : arc_length,
                      'reconstruction_time' : reconstruction_time})
    # append dataframe 
    MORData_df = MORData_df.append(df)        
    
MORData_df.to_csv(workDir+'/morData_summaryStats_%s.csv' %(model))